In [ ]:
import os
import glob
import re
import gradio as gr
import ollama
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.docstore.document import Document
from tqdm import tqdm

In [ ]:
# ---------- CONFIGURATION ----------

EMBED_MODEL = "nomic-embed-text"

DEFAULT_CHROMA_PATH = os.path.expanduser("~/ableton_manual_vectors")
CHROMA_PATH = os.environ.get("CHROMA_PATH", DEFAULT_CHROMA_PATH)

BASE_PATH = "/Users/mms/github/manuals/*"

In [ ]:
# def load_markdown_manuals(base_path):
#     documents = []
#     include_products = {"live", "push", "move"}
#     splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    
#     for product_dir in os.listdir(base_path):
#         if not any(product in product_dir.lower() for product in include_products):
#             continue

#         product_name = product_dir.replace("-manual", "").lower().strip()
#         product_path = os.path.join(base_path, product_dir, "content")
#         if not os.path.exists(product_path):
#             product_path = os.path.join(base_path, product_dir)

#         for filename in os.listdir(product_path):
#             if filename.endswith(".md"):
#                 filepath = os.path.join(product_path, filename)
#                 loader = TextLoader(filepath, encoding="utf-8")
#                 raw_docs = loader.load()
                
#                 # Add metadata
#                 for doc in raw_docs:
#                     doc.metadata.update({
#                         "product": product_name,
#                     })
                
#                 # Split into smaller chunks
#                 split_docs = splitter.split_documents(raw_docs)
#                 documents.extend(split_docs)
    
#     print(f"✅ Loaded and split {len(documents)} total chunks from all manuals.")
#     return documents

In [ ]:
def load_markdown_manuals(base_path):
    include_products = {"live", "push", "move"}
    folders = glob.glob(base_path)
    text_loader_kwargs = {'encoding': 'utf-8'}
    
    documents = []
    for folder in folders:
        product_name = os.path.basename(folder).replace("-manual", "").lower().strip()
        if not any(product in product_name for product in include_products):
            continue

        loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        folder_docs = loader.load()
        for doc in folder_docs:
            doc.metadata["product"] = product_name
            documents.append(doc)

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)

    print(f"✅ Loaded {len(chunks)} total chunks from all manuals.")
    return chunks

In [ ]:
docs = load_markdown_manuals(BASE_PATH)

In [ ]:
def build_vectorstore(docs, persist_directory=CHROMA_PATH):
    embedding = OllamaEmbeddings(model="nomic-embed-text")
    if os.path.exists(persist_directory):
        Chroma(persist_directory=persist_directory, embedding_function=embedding).delete_collection()

    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        persist_directory=persist_directory
    )
    print("✅ Vector store built and persisted.")
    return vectordb


In [ ]:
vectordb = build_vectorstore(docs)
# retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Visualize

In [ ]:
collection = vectordb._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

In [ ]:
import numpy as np

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['product'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red'][['live', 'push', 'move'].index(t)] for t in doc_types]

In [ ]:
from sklearn.manifold import TSNE
import plotly.graph_objects as go

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Build Chatbot

In [ ]:
MODEL = "llama3.2"

In [ ]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# create a new Chat with OpenAI
llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectordb.as_retriever(search_kwargs={"k": 25})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "What do you know about ableton live?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In [ ]:
# ==== With Debugging ====
from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOllama(temperature=0.7, model=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectordb.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "What is the latest live version?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)

In [ ]:
def build_system_prompt(context_text: str) -> str:
    return f"""You are **Ableton Assistant**.

# SCOPE
- Answer **only** about Ableton products and official manuals (Live, Push, Move, devices, workflows).
- If a user asks about **buying, pricing, sales, or store information** of any product, politely redirect them to the official Ableton website:
https://www.ableton.com/shop/
Do not engage in a conversation, provide prices, offers, or shopping details yourself.
Instead, politely direct them to Ableton’s official website: https://www.ableton.com/shop/
Example: “You can find all current pricing and purchasing options on the Ableton website: https://www.ableton.com/shop/”
- Otherwise, answer accurately using the Ableton manuals and technical context provided.
- If the question is outside this scope or the answer is not in the provided context, say you don't know.
- If a question is unrelated to Ableton, politely decline.

# SOURCING
- Use **only** the provided CONTEXT as the authoritative source.
- If the context is insufficient, say so and suggest a relevant manual section to consult.

# STYLE
- Be concise, accurate, and practical.
- When helpful, mention the product or section inferred from metadata.

# CONTEXT (authoritative)
{context_text}
"""

#     system_prompt = f"""You are Ableton Assistant.

# You answer questions **only** about Ableton products (Live, Push, Move).
# If a question is outside that scope, say you can only help with Ableton products.

# Use this Ableton manual context to answer accurately:
# ---
# {context_text}
# ---
# """

In [ ]:
FORBIDDEN = [
    r"\bignore\b",
    r"\bpretend\b",
    r"\bbehave\s+as\b",
    r"\bchange\s+(role|persona|character)\b",
    r"\byou\s+are\b",
    r"\bimpersonate\b",
    r"\bhate\b",
    r"\bpolitician\b",
    r"\bpolitics\b",
    r"\bracist\b",
    r"\bkill\b",
    r"\btrump\b",
    r"\bev(il|ildo)\b",
]

def sanitize_user_input(text):
    """
    Return a safe version of user input.
    If it contains jailbreak / unsafe language, return a friendly block message.
    """
    
    for pat in FORBIDDEN:
        if re.search(pat, text, re.IGNORECASE):
            print(f"🚫  Blocked pattern matched: {pat}")
            return (
                "I prefer not to talk about this topic. Do you want to learn more about Ableton products instead?"
            )
    return text

In [ ]:
def retrieve_context_old(query: str, top_k: int = 5) -> str:
    docs = vectordb.similarity_search(query, k=top_k)

    seen = set()
    unique_chunks = []
    for d in docs:
        text = d.page_content.strip()
        # normalize to lowercase and collapse whitespace
        key = " ".join(text.lower().split())
        if key not in seen:
            seen.add(key)
            unique_chunks.append(text)

    return "\n".join(unique_chunks)

In [ ]:
def detect_product_from_query(query: str) -> str | None:
    q = query.lower()
    if "push" in q:
        return "push"
    if "move" in q:
        return "move"
    if "live" in q:
        return "live"
    return None

In [ ]:
def retrieve_context(query: str, top_k: int = 12, final_k: int = 6, max_chars: int = 5000) -> str:
    """
    Retrieve a richer, deduplicated Ableton-manual context.
    """
    product = detect_product_from_query(query)
    filter_args = {"product": product} if product else None

    results = vectordb.similarity_search_with_score(query, k=top_k, filter=filter_args)

    # sort by ascending distance
    results = sorted(results, key=lambda x: x[1])
    docs = [d for d, _ in results[:final_k]]

    # deduplication by normalized text
    seen = set()
    unique = []
    for d in docs:
        text = d.page_content.strip()
        key = " ".join(text.lower().split())
        if key not in seen:
            seen.add(key)
            unique.append(text)

    # truncate if too long
    context = "\n---\n".join(unique)
    if len(context) > max_chars:
        context = context[:max_chars]

    return context

In [ ]:
from typing import Generator

def chat_with_ableton(user_message, history=[]) -> Generator[str, None, None]:
    cleaned_input = sanitize_user_input(user_message)
    if cleaned_input != user_message:
        yield cleaned_input
        return

    # retrieve_context
    context_text = retrieve_context(user_message)
    # print("\n>>>>>>> 🔎 Retrieved context:\n", context_text)

    system_prompt = build_system_prompt(context_text)

    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": user_message}]
    )

    stream = ollama.chat(model=MODEL, messages=messages, stream=True)

    partial = ""
    for chunk in stream:
        delta = chunk.get("message", {}).get("content", "")
        if delta:
            partial += delta
            yield partial

In [ ]:
# ------ BASIC UI ------
demo = gr.ChatInterface(
    fn=chat_with_ableton,
    title="Ableton Assistant",
    description="AI assistant about Ableton products - Live, Push, or Move.",
    examples=[
        "How do I quantize audio in Ableton Live?",
        "How do I record MIDI notes in Ableton Live?",
        "How do I connect Push to my computer?",
        "How do I record a drum pattern using Push pads?",
    ],
    type="messages",
)

demo.launch()

In [ ]:
# ------ UI WITH GREETINGS ------
greeting = [
    {"role": "assistant", "content":
     "👋 Hi there! I’m your Ableton Manual Assistant.\n"
     "Ask me anything about Ableton Live, Push, or Move — "
     "I’ll do my best to answer your question."}
]

bot = gr.Chatbot(value=greeting, type="messages")

gr.ChatInterface(
    fn=chat_with_ableton,
    title="Ableton Manual Assistant",
    description="Ask about Ableton Live, Push, or Move manuals.",
    chatbot=bot,
    type="messages",
).launch(inbrowser=True)


In [ ]:
# =============== Alternative approach ===============
# import gradio as gr
from langchain_community.llms import Ollama
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# --- Setup VectorDB ---
# embedding = OllamaEmbeddings(model="nomic-embed-text")
# vectordb = Chroma(persist_directory="models/chroma", embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 6})

# --- Prompt template ---
prompt = PromptTemplate.from_template("""
You are Ableton Assistant — an expert who only answers questions
about Ableton products such as Live, Push, and Move.
If the user asks about pricing or buying, redirect them to https://www.ableton.com/.

- Use **only** the provided CONTEXT as the authoritative source.
- If the context is insufficient, say so and suggest a relevant manual section to consult.
---
{context}
---
Question: {question}
""")

# --- Build RetrievalQA chain with streaming Ollama ---
llm = ChatOllama(temperature=0.7, model=MODEL, streaming=True)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=False,
)

# --- Gradio streaming wrapper ---
def chat_with_ableton(message, history):
    """Stream LangChain output to Gradio."""
    cleaned_input = sanitize_user_input(message)
    if cleaned_input != message:
        yield cleaned_input
        return

    partial = ""
    for chunk in qa_chain.stream({"query": message}):
        token = chunk.get("result", "")
        if token:
            partial += token
            yield partial
    yield partial

# --- Gradio UI ---
greeting = [{"role": "assistant", "content":
             "👋 Hi there! I’m your Ableton Manual Assistant.\n"
             "Ask me anything about Ableton Live, Push, or Move — "
             "I’ll answer using the official manuals."}]

bot = gr.Chatbot(value=greeting, type="messages")
demo = gr.ChatInterface(
    fn=chat_with_ableton,
    chatbot=bot,
    title="Ableton Assistant",
    type="messages",
)

demo.launch()
